In [1]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import urllib.request
import re
import csv

In [2]:
#with open('synth_top200.json') as s:
#    data = json.load(s)

In [3]:
#print(data)

In [2]:
resp = requests.get("http://www.vintagesynth.com/misc/virusa.php")
soup = BeautifulSoup(resp.text, 'html.parser')
scripts = json.loads(soup.find_all('script')[5].get_text())

pattern = r"vsTermMenuLink':.{'((\w+-)+(\w+))'"
found = re.findall(pattern, str(scripts))
api_key = found[0][0]
#print(api_key)
#print(scripts)
synth_brands = scripts['vsTermMenuLink'][api_key]['terms']
#print(synth_brands)

In [ ]:
#print(scripts)

In [9]:
synth_models_all = []
key_check = []
for brand_key in synth_brands.keys():
    synth_models = scripts['vsTermMenuLink'][api_key]['nodes']
    #print(synth_models)
    for fm_key, mod in synth_models.items():
        for md_key, m in list(mod.items()):
            #print('m: ', m)
            if m['key'] not in key_check:
                key_check.append(m['key'])
                model = {}
                model['firm'] = synth_brands[fm_key]['title']
                model['model_name'] = m['title']
                model['mod_key'] = m['key']
                model['url'] = m['url']

                synth_models_all.append(model)
            else:
                pass

In [10]:
print(len(synth_models_all))
print(synth_models_all[:5])

765
[{'firm': 'Korg', 'model_name': '01/W', 'mod_key': '11745', 'url': 'http://www.vintagesynth.com/korg/01w.php'}, {'firm': 'Korg', 'model_name': '707', 'mod_key': '11153', 'url': 'http://www.vintagesynth.com/korg/korg_707.php'}, {'firm': 'Korg', 'model_name': '770', 'mod_key': '11524', 'url': 'http://www.vintagesynth.com/korg/770.php'}, {'firm': 'Korg', 'model_name': '900PS Preset Synthesizer', 'mod_key': '11354', 'url': 'http://www.vintagesynth.com/korg/900ps.php'}, {'firm': 'Korg', 'model_name': 'DDD-1', 'mod_key': '11187', 'url': 'http://www.vintagesynth.com/korg/ddd1.php'}]


[{'firm': 'Roland', 'model_name': 'JX-8P', 'mod_key': '11364', 'url': 'http://www.vintagesynth.com/roland/jx8p.php'}, {'firm': 'Roland', 'model_name': 'MC-09', 'mod_key': '11482', 'url': 'http://www.vintagesynth.com/roland/mc09.php'}]


In [12]:
def cleanup_specs(specs):
    spec_keys = list(specs.keys())

    if 'Date Produced' in spec_keys:
        try:
            model['release_year'] = int(specifications['Date Produced'][:4])
        except:
            model['release_year'] = 'Please enter manually'
        try:
            if int(specifications['Date Produced'][-4:]) == model['release_year']:
                model['end_year'] = 'n/a'
            else:
                model['end_year'] = int(specifications['Date Produced'][-4:])
        except:
            model['end_year'] = 'n/a'
    else:
        model['release_year'] = 'n/a'
        model['end_year'] = 'n/a'
        
    if 'polyphony' in str(spec_keys).lower():
        if 'mono' in str(specs).lower():
            model['poly'] = 0
        else:
            model['poly'] = 1
    else:
        model['poly'] = 0
        
    if 'analog' in str(specs).lower() or 'oscillator' in str(specs).lower() or 'filter' in str(specs).lower():
        model['digital'] = 0
        model['analog'] = 1
    elif 'digital' in str(specs).lower() or model['poly']==1:
        model['digital'] = 1
        model['analog'] = 0
    else:
        model['digital'] = 'unsure'
        model['analog'] = 'unsure'
    
    if 'Keyboard' in spec_keys:
        if specs['Keyboard'] == 'None':
            model['keyboard'] = 0
        else:
            model['keyboard'] = 1
    else:
        model['keyboard'] = 0
    
    if 'Memory' in spec_keys:
        if specs['Memory'] == 'None':
            model['memory'] = 0
        else:
            model['memory'] = 1
    else:
        model['memory'] = 0

In [34]:
data = synth_models_all
for model in synth_models_all:

    resp = requests.get(model['url'])
    soup = BeautifulSoup(resp.text, 'html.parser')
    #print('soup: ===============\n',soup)
    all_scripts = soup.find_all('script')[5].get_text()
    
    # return ratings (user & VSE)
    pattern = re.compile(r'Cspan\\u003E((.)*)\\u003C\\/span', re.MULTILINE | re.DOTALL)
    script = soup.find("script", text=pattern)
    if script:
        match = pattern.search(soup.text)
        if match:
            user_rating = match.group(1)
    rt = json.loads(all_scripts)
    #print(list(rt['fiveStarts'].values())[1])
    try:
        vse_rating = float(list(rt['fiveStarts'].values())[1]['value'])
    except:
        vse_rating = 'n/a'
    model['user_rating'] = float(user_rating)
    model['vse_rating'] = vse_rating
    
    # return dictionary of specifications
    specifications = {}
    for div in soup.findAll('div', {'class': 'double-field-extended'}):
        specifications[div.find('span').attrs['title']] = div.find('span',{'class':'specification-value'}).get_text()
    
    print('================= specifications: \n', specifications, '\n====================================')
    cleanup_specs(specifications)
    model['specifications'] = specifications
    
    
    #print('=== model w/ ratings: ', model, ' ===')
    # return raw text
    text_part = ""
    for sect in soup.findAll('section', {'class': 'block-entity-fieldnodebody'}):
        for p in sect.findAll('p'):
            text_part+=str(p.get_text())
    model['raw_text'] = text_part
    #print('raw text: ==================\n', text, '\n====================================')
    
    #break
    data.append(model)

#print(model)

================= specifications: 
 {'Polyphony': '32 voices (16-part multitimbral)', 'Oscillators': '48 MB ROM using AI2 synthesis system (digital). 32 sounds, 32 oscillators (single mode); 16 sounds, 32 oscillators (double mode).', 'Sampler': 'None.', 'Effects': '2 Multi FX processors with 47 Digital multi-effects: reverb, delay, overdrive, EQ, chorus, rotary speaker, and more.', 'LFO': '2 LFOs', 'Filter': 'VDF: Digital Lowpass Filter, velocity sensitive (non-resonant)', 'VCA': 'VDA: Digital Amplifier with 4-stage ADSR envelope generators', 'Keyboard': '01/W, 01/WFD: 61 keys.01/WPro: 76 Keys.01/WProX: 88 weighted keys.All keyboards feature Velocity, Aftertouch, Multi, Layer, Split modes.', 'Sequencer': '16 tracks, 16 instruments, 48,000 notes (FD & Pro version) or 7,000 notes (01/W), 100 patterns, 10 songs. Realtime and Step-editor.', 'Arpeggiator': 'None.', 'Memory': '200 user programs, 200 user combis; Floppy: 3.5" 720 2DD (programs, combis, drum-kit, global, sequencer and midi dat

================= specifications: 
 {'Polyphony': '10 voices (parts)', 'Multitimbral': '10 Parts (4 synthesizer parts, 2 audio in parts, 2 hi-hat parts, 1 crash cymbal part, 1 hand clap part)', 'Memory': '256 Patterns, 16 songs', 'Patterns': '64 steps maximum per part, motion sequence, 1 parameter per part, 64 events', 'Songs': '256 patterns maximum per song, event recording 35,700 events maximum', 'Keyboard': 'None (16 touch pads)', 'Effects': 'Delay (normal, motion sequence, tempo delay)', 'Control': 'MIDI', 'Date Produced': '1999'} 
================= specifications: 
 {'Polyphony': '12 parts (One Shot = 9 parts, Slice = 1 part, Audio In = 1 part)', 'Sampler': '32kHz, Up to 95 seconds (mono) / 47.5 seconds (stereo). 100 mono or 50 stereo samples can be stored.', 'Patterns': '128 user patterns; Up to 256 per song', 'Songs': '16 internal song memories', 'Effects': '11 types (reverb and flanger/chorus, phaser, ring modulator, pitch shifter, compressor, distortion, decimator, isolator, r

================= specifications: 
 {'Polyphony': '2 - Duophonic', 'Oscillators': "VCO x2: Scale (64', 32', 16', 8', 4', 2'), Waveform (triangle, square, sawtooth, pulse, PWM), Pink Noise, White Noise; SUB OSC x2: Scale (32', 16', 8', 4', 2')", 'LFO': 'LFO x2: PWM + Vibrato', 'Filter': 'VCF x4: Traveler (High Pass, Low Pass)', 'VCA': 'VCA x2 + ENVELOPE GENERATOR x2: Attack Time, Singing Level, Range Selector, Mode Selector', 'Keyboard': '44 Keys', 'Memory': 'None', 'Control': 'None', 'Date Produced': '1975'} 
================= specifications: 
 {'Polyphony': '62 voices', 'Multitimbral': '16 parts', 'Oscillators': '2 oscillators', 'Waveforms': '642 multi-sampled waveforms 929 drumsamples', 'VCF': 'Digital multi-mode', 'LFO': '2 LFO per oscillator, 4 per patch', 'VCA': '3 6-point envelopes (Osc, Filter, Amp)', 'Control': 'MIDI In/Out, USB MIDI', 'Arpeggiator': '2 programmable (256 presets)', 'Effects': '89 effect types, 3 per patch', 'Keyboard': '25 keys w/ velocity', 'Memory': '640 Prog

================= specifications: 
 {'Polyphony': 'Full 60 notes', 'Oscillators': '1 VCO per note', 'LFO': '1 LFO with Vibrato Depth, Vibrato Speed', 'Filter': '1 resonant low pass, 1 resonant high pass', 'Envelope': 'Attack, Decay, Sustain', 'Effects': 'Portamento (Glide)', 'Keyboard': '60 Keys', 'Memory': 'None', 'Control': 'Traveler Expression, Damper pedal control, Glide pedal control', 'Date Produced': '1976 - 1979'} 
================= specifications: 
 {'Polyphony': 'Full 48 notes', 'Oscillators': "3 VCO's per note", 'LFO': 'None', 'Filter': 'Bass (low pass), Treble (high pass)', 'Envelope': 'Attack, Sustain', 'Effects': 'Phaser', 'Keyboard': '48 Keys', 'Memory': 'None', 'Control': 'Expression Pedal', 'Date Produced': '1976 - 1979'} 
================= specifications: 
 {'Polyphony': '6 Voices', 'Oscillators': '1 VCO per voice (saw, PW, PWM) + 1 sub-oscillator per voice', 'LFO': '1 LFO assignable to VCA,VCF or VCO', 'Filter': 'Low-pass only, self-oscillates at high resonance. ADSR

================= specifications: 
 {'Polyphony': 'Triton & LE - 62 voices (68 with MOSS)Triton Rack - 60 voices (66 with MOSS)Triton Studio & Extreme - 120 voices', 'Tone Generator': 'HI synthesis system; 48 kHz sampling frequency, 32 Mbyte PCM ROM, 425 multi-samples + 413 drum samples ', 'Sound Source': '62 oscillators in single mode/ 31 voices, 62 oscillators in double mode. (60 and 30 osc. in Rack module)', 'Sampler': '16 bit, 48 kHz stereo/mono sampling, 16 MB memory standard, expandable to 64 MB. Maximum of 1,000 multi-samples / 4,000 samples. Up to 128 samples can be assigned to a multi-sample. AIFF, WAVE, AKAI (S1000/S3000) and Trinity sample data can be loaded. (Triton cannot load Trinity format data from TFD-1S, TFD-2S, TFD-3S and TFD-4S , since they use data-compressed data.)', 'Memory': '640 programs, 512 combinations, 64 drum programs, GM Level 2 - 256 programs + 9 drum set; External Floppy Disk Drive: 3.5 inch 2DD/2HD', 'Keyboard': 'All models offered: 88 weighted keys, 7

...

================= specifications: 
 {'Polyphony': 'Monophonic', 'Oscillators': '1 VCO', 'LFO': 'none', 'Filter': '18dB 2-pole filter with cutoff, resonance, env mod and decay', 'Sequencer': 'Yes', 'Keyboard': 'none', 'Memory': 'none', 'Control': 'MIDI, Roland DIN Sync, CV / Gate', 'Date Produced': '1997'} 
================= specifications: 
 {'Polyphony': 'Mono/Duo-phonic', 'Oscillators': "2 VCO's (2 sawtooth, 2 square wave, and 1 triangle) plus 2 sub-oscillators", 'LFO': '1 with sample-and-hold', 'Filter': '1', 'VCA': '1 VCA, 2 EGs- AR and ASDR', 'Keyboard': '37 keys', 'Memory': 'None', 'Control': 'CV/Gate', 'Date Produced': '1976-81'} 
================= specifications: 
 {'Polyphony': 'Monophonic', 'Oscillators': '1 VCO plus white noise generator', 'LFO': '1 LFO, Sine & Square waveforms, can modulate VCA or VCF', 'Filter': '1 VCF ', 'VCA': '1 VCA with ADSR ', 'Keyboard': '37 keys', 'Memory': 'None', 'Control': 'CV/Gate ', 'Date Produced': '1981 '} 
================= specifications: 


================= specifications: 
 {'Polyphony': 'Between 25 to 110 voices, depending on the complexity of the patch', 'Multitimbral': '16 part', 'Oscillators': '3 oscillators with one sub oscillator per voice', 'Waveforms': 'Traditional VA, Wavetable, Graintable, HyperSaw, and Formant', 'LFO': '3 LFOs with 68 waveforms', 'Filter': '2 multi-mode filters (lowpass, highpass, bandpass, bandstop) with an optional saturation module and an Analog Filter modeled after the Moog cascade filter with 6 to 24 dB/oct slopes', 'Envelope': '2 ADSTR envelopes', 'Effects': '129 parallel effects: delay, reverb, distortion/lo-fi algorithms, phaser, chorus/flanger, character control, ring modulator/shifter, EQ, vocoder', 'Sequencer': 'None', 'Arpeggiator': '32 steps with global control for swing/shuffle and note lengths', 'Keyboard': '37 keys with velocity and aftertouch', 'Memory': '1024 RAM and 3328 ROM sounds', 'Control': 'MIDI In/Out/Thru and USB', 'Date Produced': 'TI Polar - 2005TI2 Polar 2009'} 
=

================= specifications: 
 {'Polyphony': '6 voices', 'Oscillators': '2 DCOs per voice; DWGS: 8 sampled digital waveforms', 'LFO': 'YES', 'Filter': 'Analog Low-pass resonant filter with ADBSSR 6-parameter envelope generator', 'VCA': 'ADBSSR 6-parameter envelope generator', 'Keyboard': '61 keys (no velocity or aftertouch)', 'Memory': '64 patches', 'Control': 'MIDI', 'Date Produced': '1985'} 
================= specifications: 
 {'Polyphony': '8 voices', 'Oscillators': '2 DWGSs per voice; 16 sampled digital waveforms (including saw, sqr and sin)', 'LFO': 'YES', 'Filter': 'Analog Low-pass resonant filter with ADBSSR 6-parameter envelope generator', 'VCA': 'ADBSSR 6-parameter envelope generator', 'Keyboard': '61 keys (with velocity and aftertouch)', 'Arpeg/Seq': '64 note arpeggiator', 'Effects': 'Digital Delay', 'Control': 'MIDI', 'Date Produced': '1985-87'} 
================= specifications: 
 {'Polyphony': '2 voices', 'Waveforms': 'Sawtooth, square, tri, external audio-in', 'Memor

================= specifications: 
 {'Polyphony': 'MS-20, Polysix: 32 voices.Wavestation, M1: 256 voices.Mono/Poly: 128 voices.', 'Oscillators': "MS-20: 2 VCOs'Polysix: 1 VCO per voiceWavestation: 484 PCM waveformsM1: 950 PCM Waveforms", 'Filter': 'MS-20: Independent High Pass and Low Pass filters.Polysix: Lowpass filter.Wavestation: Lowpass filters.', 'Effects': 'MS-20: Portamento.Polysix: Chorus, Ensemble, Phase.MDE-X Effects: Compressors, Limiters, EQ, exciters, decimator, flanger, phaser, chorus, delay, reverbs, overdrive, wah-wah.Mono/Poly: Two multi-effect processors.M1: 34 Types of on-board effects.', 'Keyboard': 'Virtual (MS-20 controller has 37 velocity sensitive mini-keys)  ', 'Memory': 'MS-20: 64 programs.Polysix: 32 programs.Wavestation: More than 1,500 preset sounds (15 ROM banks and 3 RAM banks).Mono/Poly: 256 Programs.M1: 3,100 presets sounds; 400 user locations.', 'Arpeg/Seq': 'Polysix, Mono/Poly: Up, Down, Up/Down (Tempo/MIDI sync)', 'Macintosh': 'Apple G4, 1.25GHz or 

IndexError: list index out of range

In [40]:
print(data[289:291])

[{'firm': 'Roland', 'model_name': 'JX-8P', 'mod_key': '11364', 'url': 'http://www.vintagesynth.com/roland/jx8p.php', 'user_rating': 4.19, 'vse_rating': 4.0, 'release_year': 1985, 'end_year': 'n/a', 'poly': 1, 'digital': 0, 'analog': 1, 'keyboard': 1, 'memory': 1, 'specifications': {'Polyphony': '6 voices', 'Oscillators': "2 DCO's per voice (12 oscillators)", 'Memory': '64 patches + External memory cartridges', 'Effects': 'Portamento', 'VCF': 'standard VCF', 'VCA': 'ADSR envelope', 'Arpeg/Seq': 'None', 'Keyboard': '61 note keyboard with velocity and aftertouch', 'Control': 'MIDI In/Out/thru', 'Date Produced': '1985'}, 'raw_text': "\xa0\xa0The JX-8P is a decent analog polysynth. It has 6 voices of polyphony, great MIDI implementation and patch storage as well as an external memory cartridge slot. Though it is far more advanced than its predecessor, the JX-3P, the JX-8P has its drawbacks. Hands-on programming is sacrificed and reduced to assigning the parameter you want to tweak to a data

In [15]:
# return a unique list of keys
ls_of_keys = []
for d in data:
    for ks in d:
        if ks not in ls_of_keys:
            ls_of_keys.append(ks)

In [16]:
print(ls_of_keys)

['firm', 'model_name', 'mod_key', 'url', 'user_rating', 'vse_rating', 'release_year', 'end_year', 'poly', 'digital', 'analog', 'keyboard', 'memory', 'specifications', 'raw_text']


In [41]:
print(data[290])

{'firm': 'Roland', 'model_name': 'MC-09', 'mod_key': '11482', 'url': 'http://www.vintagesynth.com/roland/mc09.php', 'user_rating': 3.1, 'vse_rating': 3.0, 'release_year': 2002, 'end_year': 'n/a', 'poly': 1, 'digital': 0, 'analog': 1, 'keyboard': 0, 'memory': 1, 'specifications': {'Polyphony': 'Lead: 1, Bass: 1, Rhythm: 3', 'Oscillators': 'Preset Tones: 128 Lead, 128 Bass, 10 Rhythm', 'Sequencer': '32 Steps, Sixteenth note ResolutionPreset Patterns: 40 Lead, 60 Bass, 100 Rhythm, 30 Effect;User Patterns: 20', 'Memory': 'SmartMedia Storage', 'Effects': '30: Filter, Isolator, Phaser, Slicer, Lead Sound Group: Overdrive, Distortion, Phaser, Slicer + Ring Modulator', 'LFO': 'Triangle, Square, Sawtooth, Sample and Hold', 'Filter': 'Low pass, Band pass, high pass', 'VCA': 'ADSR', 'Keyboard': 'None', 'Control': 'MIDI In/Out', 'Date Produced': '2002'}, 'raw_text': 'The MC-09 PhraseLab is a creative tool for producing synth and drum \nloops - and emulating the TB-303.\nCombining an analog modelin

In [44]:
# To CSV
csv_columns = ls_of_keys
x=1
csv_file = "new_Coded_Models_by_manufacturer_resurgence.csv"

try:
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for d in data:
            if 'raw_text' in d.keys():
                if '\u2032' in str(d['raw_text']):
                    d['raw_text'] = d['raw_text'].replace('\u2032','feet')
            writer.writerow(d)
            print(x, " inserted --- ")
            x+=1
except IOError:
    print("I/O error")
    pass

1  inserted --- 
2  inserted --- 
3  inserted --- 
4  inserted --- 
5  inserted --- 
6  inserted --- 
7  inserted --- 
8  inserted --- 
9  inserted --- 
10  inserted --- 
11  inserted --- 
12  inserted --- 
13  inserted --- 
14  inserted --- 
15  inserted --- 
16  inserted --- 
17  inserted --- 
18  inserted --- 
19  inserted --- 
20  inserted --- 
21  inserted --- 
22  inserted --- 
23  inserted --- 
24  inserted --- 
25  inserted --- 
26  inserted --- 
27  inserted --- 
28  inserted --- 
29  inserted --- 
30  inserted --- 
31  inserted --- 
32  inserted --- 
33  inserted --- 
34  inserted --- 
35  inserted --- 
36  inserted --- 
37  inserted --- 
38  inserted --- 
39  inserted --- 
40  inserted --- 
41  inserted --- 
42  inserted --- 
43  inserted --- 
44  inserted --- 
45  inserted --- 
46  inserted --- 
47  inserted --- 
48  inserted --- 
49  inserted --- 
50  inserted --- 
51  inserted --- 
52  inserted --- 
53  inserted --- 
54  inserted --- 
55  inserted --- 
56  inserted --- 
5

438  inserted --- 
439  inserted --- 
440  inserted --- 
441  inserted --- 
442  inserted --- 
443  inserted --- 
444  inserted --- 
445  inserted --- 
446  inserted --- 
447  inserted --- 
448  inserted --- 
449  inserted --- 
450  inserted --- 
451  inserted --- 
452  inserted --- 
453  inserted --- 
454  inserted --- 
455  inserted --- 
456  inserted --- 
457  inserted --- 
458  inserted --- 
459  inserted --- 
460  inserted --- 
461  inserted --- 
462  inserted --- 
463  inserted --- 
464  inserted --- 
465  inserted --- 
466  inserted --- 
467  inserted --- 
468  inserted --- 
469  inserted --- 
470  inserted --- 
471  inserted --- 
472  inserted --- 
473  inserted --- 
474  inserted --- 
475  inserted --- 
476  inserted --- 
477  inserted --- 
478  inserted --- 
479  inserted --- 
480  inserted --- 
481  inserted --- 
482  inserted --- 
483  inserted --- 
484  inserted --- 
485  inserted --- 
486  inserted --- 
487  inserted --- 
488  inserted --- 
489  inserted --- 
490  inserte

938  inserted --- 
939  inserted --- 
940  inserted --- 
941  inserted --- 
942  inserted --- 
943  inserted --- 
944  inserted --- 
945  inserted --- 
946  inserted --- 
947  inserted --- 
948  inserted --- 
949  inserted --- 
950  inserted --- 
951  inserted --- 
952  inserted --- 
953  inserted --- 
954  inserted --- 
955  inserted --- 
956  inserted --- 
957  inserted --- 
958  inserted --- 
959  inserted --- 
960  inserted --- 
961  inserted --- 
962  inserted --- 
963  inserted --- 
964  inserted --- 
965  inserted --- 
966  inserted --- 
967  inserted --- 
968  inserted --- 
969  inserted --- 
970  inserted --- 
971  inserted --- 
972  inserted --- 
973  inserted --- 
974  inserted --- 
975  inserted --- 
976  inserted --- 
977  inserted --- 
978  inserted --- 
979  inserted --- 
980  inserted --- 
981  inserted --- 
982  inserted --- 
983  inserted --- 
984  inserted --- 
985  inserted --- 
986  inserted --- 
987  inserted --- 
988  inserted --- 
989  inserted --- 
990  inserte

1437  inserted --- 
1438  inserted --- 
1439  inserted --- 
1440  inserted --- 
1441  inserted --- 
1442  inserted --- 
1443  inserted --- 
1444  inserted --- 
1445  inserted --- 
1446  inserted --- 
1447  inserted --- 
1448  inserted --- 
1449  inserted --- 
1450  inserted --- 
1451  inserted --- 
1452  inserted --- 
1453  inserted --- 
1454  inserted --- 
1455  inserted --- 
1456  inserted --- 
1457  inserted --- 
1458  inserted --- 
1459  inserted --- 
1460  inserted --- 
1461  inserted --- 
1462  inserted --- 
1463  inserted --- 
1464  inserted --- 
1465  inserted --- 
1466  inserted --- 
1467  inserted --- 
1468  inserted --- 
1469  inserted --- 
1470  inserted --- 
1471  inserted --- 
1472  inserted --- 
1473  inserted --- 
1474  inserted --- 
1475  inserted --- 
1476  inserted --- 
1477  inserted --- 
1478  inserted --- 
1479  inserted --- 
1480  inserted --- 
1481  inserted --- 
1482  inserted --- 
1483  inserted --- 
1484  inserted --- 
1485  inserted --- 
1486  inserted --- 


1848  inserted --- 
1849  inserted --- 
1850  inserted --- 
1851  inserted --- 
1852  inserted --- 
1853  inserted --- 
1854  inserted --- 
1855  inserted --- 
1856  inserted --- 
1857  inserted --- 
1858  inserted --- 
1859  inserted --- 
1860  inserted --- 
1861  inserted --- 
1862  inserted --- 
1863  inserted --- 
1864  inserted --- 
1865  inserted --- 
1866  inserted --- 
1867  inserted --- 
1868  inserted --- 
1869  inserted --- 
1870  inserted --- 
1871  inserted --- 
1872  inserted --- 
1873  inserted --- 
1874  inserted --- 
1875  inserted --- 
1876  inserted --- 
1877  inserted --- 
1878  inserted --- 
1879  inserted --- 
1880  inserted --- 
1881  inserted --- 
1882  inserted --- 
1883  inserted --- 
1884  inserted --- 
1885  inserted --- 
1886  inserted --- 
1887  inserted --- 
1888  inserted --- 
1889  inserted --- 
1890  inserted --- 
1891  inserted --- 
1892  inserted --- 
1893  inserted --- 
1894  inserted --- 
1895  inserted --- 
1896  inserted --- 
1897  inserted --- 


2436  inserted --- 
2437  inserted --- 
2438  inserted --- 
2439  inserted --- 
2440  inserted --- 
2441  inserted --- 
2442  inserted --- 
2443  inserted --- 
2444  inserted --- 
2445  inserted --- 
2446  inserted --- 
2447  inserted --- 
2448  inserted --- 
2449  inserted --- 
2450  inserted --- 
2451  inserted --- 
2452  inserted --- 
2453  inserted --- 
2454  inserted --- 
2455  inserted --- 
2456  inserted --- 
2457  inserted --- 
2458  inserted --- 
2459  inserted --- 
2460  inserted --- 
2461  inserted --- 
2462  inserted --- 
2463  inserted --- 
2464  inserted --- 
2465  inserted --- 
2466  inserted --- 
2467  inserted --- 
2468  inserted --- 
2469  inserted --- 
2470  inserted --- 
2471  inserted --- 
2472  inserted --- 
2473  inserted --- 
2474  inserted --- 
2475  inserted --- 
2476  inserted --- 
2477  inserted --- 
2478  inserted --- 
2479  inserted --- 
2480  inserted --- 
2481  inserted --- 
2482  inserted --- 
2483  inserted --- 
2484  inserted --- 
2485  inserted --- 


2865  inserted --- 
2866  inserted --- 
2867  inserted --- 
2868  inserted --- 
2869  inserted --- 
2870  inserted --- 
2871  inserted --- 
2872  inserted --- 
2873  inserted --- 
2874  inserted --- 
2875  inserted --- 
2876  inserted --- 
2877  inserted --- 
2878  inserted --- 
2879  inserted --- 
2880  inserted --- 
2881  inserted --- 
2882  inserted --- 
2883  inserted --- 
2884  inserted --- 
2885  inserted --- 
2886  inserted --- 
2887  inserted --- 
2888  inserted --- 
2889  inserted --- 
2890  inserted --- 
2891  inserted --- 
2892  inserted --- 
2893  inserted --- 
2894  inserted --- 
2895  inserted --- 
2896  inserted --- 
2897  inserted --- 
2898  inserted --- 
2899  inserted --- 
2900  inserted --- 
2901  inserted --- 
2902  inserted --- 
2903  inserted --- 
2904  inserted --- 
2905  inserted --- 
2906  inserted --- 
2907  inserted --- 
2908  inserted --- 
2909  inserted --- 
2910  inserted --- 
2911  inserted --- 
2912  inserted --- 
2913  inserted --- 
2914  inserted --- 


3593  inserted --- 
3594  inserted --- 
3595  inserted --- 
3596  inserted --- 
3597  inserted --- 
3598  inserted --- 
3599  inserted --- 
3600  inserted --- 
3601  inserted --- 
3602  inserted --- 
3603  inserted --- 
3604  inserted --- 
3605  inserted --- 
3606  inserted --- 
3607  inserted --- 
3608  inserted --- 
3609  inserted --- 
3610  inserted --- 
3611  inserted --- 
3612  inserted --- 
3613  inserted --- 
3614  inserted --- 
3615  inserted --- 
3616  inserted --- 
3617  inserted --- 
3618  inserted --- 
3619  inserted --- 
3620  inserted --- 
3621  inserted --- 
3622  inserted --- 
3623  inserted --- 
3624  inserted --- 
3625  inserted --- 
3626  inserted --- 
3627  inserted --- 
3628  inserted --- 
3629  inserted --- 
3630  inserted --- 
3631  inserted --- 
3632  inserted --- 
3633  inserted --- 
3634  inserted --- 
3635  inserted --- 
3636  inserted --- 
3637  inserted --- 
3638  inserted --- 
3639  inserted --- 
3640  inserted --- 
3641  inserted --- 
3642  inserted --- 
